<a href="https://colab.research.google.com/github/yashvicky/rag-trillion-dollar-demo/blob/main/Trillion_Dollar_company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu openai streamlit


In [ ]:
!pip install sentence-transformers transformers faiss-cpu


In [ ]:
docs = [
    "The next trillion-dollar company will likely dominate an industry where AI agents become indispensable, integrating deeply into workflows and consumer life.",
    "Sustainability and climate-tech may fuel the rise of the next trillion-dollar firm, driven by breakthroughs in clean energy, carbon capture, and scalable green infrastructure.",
    "The next trillion-dollar opportunity could come from biotech, where personalized medicine and gene editing converge with AI to drastically lower costs and improve outcomes.",
    "Platforms that orchestrate AI-first enterprises—where every department is agentic and self-improving—are strong contenders for trillion-dollar scale.",
    "Space economy expansion, including satellite networks, asteroid mining, and interplanetary logistics, could create trillion-dollar valuations within two decades.",
    "Trust and security will be non-negotiable; the next trillion-dollar company will win by proving it can deploy AI at scale safely, ethically, and under tight regulatory frameworks.",
    "Consumer adoption will hinge on seamless design; trillion-dollar winners will make advanced AI invisible and intuitive, like the iPhone did for mobile computing."
]


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import pipeline

# 1. Embedding model (retriever)
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Generative model (answering)
gen_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# 3. Knowledge base: your trillion-dollar company notes
docs = [
    "The next trillion-dollar company will likely dominate an industry where AI agents become indispensable, integrating deeply into workflows and consumer life.",
    "Sustainability and climate-tech may fuel the rise of the next trillion-dollar firm, driven by breakthroughs in clean energy, carbon capture, and scalable green infrastructure.",
    "The next trillion-dollar opportunity could come from biotech, where personalized medicine and gene editing converge with AI to drastically lower costs and improve outcomes.",
    "Platforms that orchestrate AI-first enterprises—where every department is agentic and self-improving—are strong contenders for trillion-dollar scale.",
    "Space economy expansion, including satellite networks, asteroid mining, and interplanetary logistics, could create trillion-dollar valuations within two decades.",
    "Trust and security will be non-negotiable; the next trillion-dollar company will win by proving it can deploy AI at scale safely, ethically, and under tight regulatory frameworks.",
    "Consumer adoption will hinge on seamless design; trillion-dollar winners will make advanced AI invisible and intuitive, like the iPhone did for mobile computing."
]

# 4. Build FAISS index
embeddings = embed_model.encode(docs, convert_to_numpy=True).astype("float32")
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# 5. RAG function
def rag_generate(question, k=3):
    # Retrieve top k passages
    q_embed = embed_model.encode([question], convert_to_numpy=True).astype("float32")
    D, I = index.search(q_embed, k=k)
    retrieved = [docs[i] for i in I[0]]

    context = " ".join(retrieved)
    prompt = f"""Use the context below to answer concisely and avoid repetition.

Context: {context}

Question: {question}
Answer in 2-3 clear sentences:
"""
    result = gen_pipeline(prompt, max_new_tokens=150, do_sample=False, truncation=True)
    return result[0]["generated_text"], retrieved


Device set to use cpu


In [ ]:
answer, retrieved = rag_generate("What challenges might the next trillion-dollar company face?")
print("💡 Answer:", answer)
print("\n📚 Context used:")
for r in retrieved:
    print("-", r)


💡 Answer: sustainability and climate-tech

📚 Context used:
- Trust and security will be non-negotiable; the next trillion-dollar company will win by proving it can deploy AI at scale safely, ethically, and under tight regulatory frameworks.
- Sustainability and climate-tech may fuel the rise of the next trillion-dollar firm, driven by breakthroughs in clean energy, carbon capture, and scalable green infrastructure.
- The next trillion-dollar company will likely dominate an industry where AI agents become indispensable, integrating deeply into workflows and consumer life.


In [ ]:
answer, retrieved = rag_generate("What challenges might the next trillion-dollar company face?")
print("🔍 Question:", "What challenges might the next trillion-dollar company face?")
print("💡 Generated Answer:\n", answer)
print("\n📚 Context used:")
for r in retrieved:
    print("-", r)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔍 Question: What challenges might the next trillion-dollar company face?
💡 Generated Answer:
 Use the context below to answer concisely and avoid repetition.
    
Context: Trust and security will be non-negotiable; the next trillion-dollar company will win by proving it can deploy AI at scale safely, ethically, and under tight regulatory frameworks. Sustainability and climate-tech may fuel the rise of the next trillion-dollar firm, driven by breakthroughs in clean energy, carbon capture, and scalable green infrastructure. The next trillion-dollar company will likely dominate an industry where AI agents become indispensable, integrating deeply into workflows and consumer life.

Question: What challenges might the next trillion-dollar company face?
Answer in 2-3 clear sentences:

The next trillion-dollar company will likely face a number of challenges.

The next trillion-dollar company will likely face a number of challenges.

The next trillion-dollar company will likely face a number of

In [ ]:
# Second test
answer2, retrieved2 = rag_generate("Which industries are most promising for the next trillion-dollar company?")
print("💡 Answer:", answer2)
print("\n📚 Context used:")
for r in retrieved2:
    print("-", r)


💡 Answer: clean energy, carbon capture, and scalable green infrastructure

📚 Context used:
- Sustainability and climate-tech may fuel the rise of the next trillion-dollar firm, driven by breakthroughs in clean energy, carbon capture, and scalable green infrastructure.
- The next trillion-dollar company will likely dominate an industry where AI agents become indispensable, integrating deeply into workflows and consumer life.
- The next trillion-dollar opportunity could come from biotech, where personalized medicine and gene editing converge with AI to drastically lower costs and improve outcomes.


In [ ]:
# Second test question
answer2, retrieved2 = rag_generate("Which industries are most promising for the next trillion-dollar company?")
print("💡 Answer:", answer2)
print("\n📚 Context used:")
for r in retrieved2:
    print("-", r)


💡 Answer: clean energy, carbon capture, and scalable green infrastructure

📚 Context used:
- Sustainability and climate-tech may fuel the rise of the next trillion-dollar firm, driven by breakthroughs in clean energy, carbon capture, and scalable green infrastructure.
- The next trillion-dollar company will likely dominate an industry where AI agents become indispensable, integrating deeply into workflows and consumer life.
- The next trillion-dollar opportunity could come from biotech, where personalized medicine and gene editing converge with AI to drastically lower costs and improve outcomes.
